In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow.keras as K
import tensorflow as tf

In [3]:
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [4]:
def preprocess_data(X, Y):
    """
    Pre-processes the the CIFAR 10 dataset for the VGG16 model

        Args:
            X is a numpy.ndarray of shape (m, 32, 32, 3)
                containing the CIFAR 10 data, where m is
                the number of data points
            Y is a numpy.ndarray of shape (m,) containing
                the CIFAR 10 labels for X

        Returns:
            X_p, Y_p where:
                X_p is a numpy.ndarray containing the preprocessed X
                Y_p is a numpy.ndarray containing the preprocessed Y
    """
    X_p = K.applications.vgg16.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)
    return X_p, Y_p

In [5]:

# load cifar 10 dataset
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()

In [6]:
# prepare data
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [7]:
# input tensor
input = K.Input(shape=(32, 32, 3))

In [8]:
# resizes images to required dimensions for VGG16
scaled_up_layer = K.layers.Lambda(lambda i: tf.image.resize_image_with_pad(
    image=i,
    target_height=48,
    target_width=48,
    method=tf.image.ResizeMethod.BILINEAR))(input)

In [9]:
# Load VGG16 without the top classification layer
base_model = K.applications.VGG16(weights='imagenet',
                                  include_top=False,
                                  input_tensor=scaled_up_layer,
                                  input_shape=(48, 48, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
# Extracts the last layer from third block of VGG16 model
last = base_model.get_layer('block3_pool').output

In [11]:
# Freezes the layers of VGG16
base_model.trainable = False


In [12]:
# Adds classification layers on top of model
layer = K.layers.GlobalAveragePooling2D()(last)
layer = K.layers.BatchNormalization()(layer)
layer = K.layers.Dense(units=256, activation='relu')(layer)
layer = K.layers.Dropout(rate=0.6)(layer)
# CIFAR 10 has 10 classes
classes = 10
output = K.layers.Dense(units=classes, activation='softmax')(layer)
model = K.Model(input, output)

In [13]:
Adam = K.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=Adam,
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
lambda (Lambda)              (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584

Train the Model

In [15]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    validation_data=(x_test, y_test),
                    epochs=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 8s 170us/sample - loss: 0.2926 - acc: 0.8976 - val_loss: 0.2021 - val_acc: 0.9213
Epoch 2/20
50000/50000 [==============================] - 7s 136us/sample - loss: 0.2236 - acc: 0.9150 - val_loss: 0.1774 - val_acc: 0.9309
Epoch 3/20
50000/50000 [==============================] - 7s 136us/sample - loss: 0.2004 - acc: 0.9223 - val_loss: 0.1650 - val_acc: 0.9356
Epoch 4/20
50000/50000 [==============================] - 7s 135us/sample - loss: 0.1870 - acc: 0.9273 - val_loss: 0.1571 - val_acc: 0.9388
Epoch 5/20
50000/50000 [==============================] - 7s 135us/sample - loss: 0.1776 - acc: 0.9310 - val_loss: 0.1514 - val_acc: 0.9409
Epoch 6/20
50000/50000 [==============================] - 7s 135us/sample - loss: 0.1708 - acc: 0.9330 - val_loss: 0.1470 - val_acc: 0.9424
Epoch 7/20
50000

In [16]:
model.save('cifar10.h5')



In [17]:
K.learning_phase = K.backend.learning_phase 

In [18]:
_, (X, Y) = K.datasets.cifar10.load_data()

In [19]:
X_p, Y_p = preprocess_data(X, Y)

In [23]:
tf.keras.models.Model() 

In [22]:
model.evaluate(X_p, Y_p, batch_size=128, verbose=1)

10000/10000 [==============================] - 1s 110us/sample - loss: 0.1245 - acc: 0.9517


[0.12447632614374161, 0.95173985]